<a href="https://colab.research.google.com/github/SwapnilAryan97/projects/blob/master/EmailSpamDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the libraries

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

import tensorflow as tf
tf.test.gpu_device_name()
from tensorflow.python.client import device_lib
device_lib.list_local_devices()
!cat /proc/meminfo

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
MemTotal:       13333552 kB
MemFree:         9466476 kB
MemAvailable:   11909652 kB
Buffers:           78392 kB
Cached:          2393516 kB
SwapCached:            0 kB
Active:          1461248 kB
Inactive:        2008828 kB
Active(anon):     841828 kB
Inactive(anon):     8552 kB
Active(file):     619420 kB
Inactive(file):  2000276 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:              1044 kB
Writeback:             0 kB
AnonPages:        998072 kB
Mapped:           532060 kB
Shmem:              9164 kB
Slab:             175372 kB
SReclaimable:     128604 kB
SUnreclaim:        46768 kB
KernelStack:        3936 kB
PageTables:         8368 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6666776 kB
Committed_AS:    3223916 kB
VmallocTotal:

Importing the dataset

In [ ]:
url = 'https://raw.githubusercontent.com/SwapnilAryan97/EmailSpamDetection/master/emails.csv'
dataset = pd.read_csv(url)
print(dataset)
row_size = dataset.shape[0]
print(row_size)

                                                   text  spam
0     Subject: naturally irresistible your corporate...     1
1     Subject: the stock trading gunslinger  fanny i...     1
2     Subject: unbelievable new homes made easy  im ...     1
3     Subject: 4 color printing special  request add...     1
4     Subject: do not have money , get software cds ...     1
...                                                 ...   ...
5723  Subject: re : research and development charges...     0
5724  Subject: re : receipts from visit  jim ,  than...     0
5725  Subject: re : enron case study update  wow ! a...     0
5726  Subject: re : interest  david ,  please , call...     0
5727  Subject: news : aurora 5 . 2 update  aurora ve...     0

[5728 rows x 2 columns]
5728


Cleaning the texts

In [ ]:
corpus = []
for i in range (0, row_size):
    review = re.sub('[^a-zA-Z]', ' ', dataset['text'][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    all_stopwords = stopwords.words('english')
    review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
    review = ' '.join(review)
    corpus.append(review)

Creating Bag of Words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = len(corpus) )
x = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, -1].values

Splitting dataset into Training and Testing set

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Predicting Test Set result

In [ ]:
y_pred = classifier.predict(x_test)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))

[[0 0]
 [0 0]
 [1 1]
 ...
 [0 0]
 [0 0]
 [1 1]]


Making Confusion Matrix & Accuracy Score

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
print('Accuracy is: {:.2f}%'.format(accuracy_score(y_test, y_pred)*100))

[[853   9]
 [  2 282]]
Accuracy is: 99.04%


Applying k-Fold Cross Validation

In [ ]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = x_train, y = y_train, cv = 10)
print('Accuracy: {:.2f}%'.format(accuracies.mean()*100))

Accuracy: 99.08%
